**Постановка**

Мы анализируем чеки (они же транзакции) покупателей супермаркета. В чеке может быть любой набор товаров.

<img src="img/transactions.png" width=300>

**Задача** 

Мы хотим найти устойчивые закономерности в наборах продуктов и использовать их например при выкладке товаров на полке или планировании рекламных акций. 

Например: 
- при покупке хлеба часто покупают молоко
- с вином часто покупают фрукты

Хрестоматийный пример - анализируя чеки из супермаркета аналитики нашли устойчивую закономерность "Покупка детского питания" => "Покупка пива".

**Определение**

Ассоциативное правило - закономерность вида $A \rightarrow B$, где A и B - множества продуктов в корзине. 

Читается правило так: Если клиент купил набор товаров A, то с высокой вероятностью купит набор продуктов B. Пример записи таких правил в таблице ниже (колонка Rule):

<img src="img/rules_example.jpg" width=300>

Для того, чтобы можно было сравнивать правила между собой, вводится несколько простых метрик.

**1) Support правила** - доля чеков, в которых присутсвтует и продукт A, и продукт B

$$supp(A \rightarrow B)=\frac{|A \cap B|}{|N|}$$

**2) Confidence правила** - доля тех, кто купил B среди тех, кто купил A

$$conf(A \rightarrow B)=\frac{supp(A \cap B)}{supp(A)}$$

**3) Lift правила** - насколько confidence правила A->B выше/ниже общей популярности продукта B

$$lift(A \rightarrow B)=\frac{conf(A \rightarrow B)}{supp(B)} = \frac{supp(A \cap B)}{supp(A) \cdot supp(B)}$$

Зачем нужен Lift: высокий confidence может объясняться просто большой популярностью продукта B, а вовсе не его хорошими дополняющими свойствами к продукту A. Метрика Lift этой особенности лишена.

Support трактуется как "устойчивость" зависимости (как часто имеем возможность подобное правило наблюдать)

Confidence трактуется как "сила" зависимости (при совпадении нужного набора продуктов как часто это правило действует)

**Общий алгоритм** поиска ассоциативных правил:

1) Найти комбинации часто встречающихся вместе продуктов (**frequent itemsets**)

2) Для отобранных множеств-кандидатов построить на них все возможные ассоциативные правила, посчитать их метрики (confidence, support), отсортировать и вывести список наилучших правил

В правиле $A \rightarrow B$:
- Левая часть (A) называется **Antecedent**
- Правая часть (B) - называется **Consequent**

Есть несколько способов представления товаров в чеке:
- денормализованный "плотный" формат - делаем табличку с 1000 колонками (по числу товаров в магазине) и отмечаем те, которые клиент купил

|Transaction_id| item1 | item2 | item3 | item4 | item5 | item6 | item7 | item8 | item9 | item10 |
|---|---|---|---|---|---|---|---|---|---|---|
|trn_1| 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 |
|trn_2| 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 |
|trn_3| 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 |
|trn_4| 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
|trn_5| 0 | 1 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 |

- денормализованный "разреженный" формат (строка с названиями/идентификаторам продукта):

|Transaction_id| items_list |
|---|---|
|trn_1| {item3, item8} |
|trn_2| {item1, item3, item6, item9} |
|trn_3| {item2, item6, item8} |
|trn_4| {item1, item4} |
|trn_5| {item2, item6, item7} |

- нормализованный - одна запись таблицы это пара (trn_id, item)

|Transaction_id | item_id |
|---|---|
|trn_1| item3 |
|trn_1| item8 |
|trn_2| item1 |
|trn_2| item3 |
|trn_2| item6 |
|trn_2| item9 |
|trn_3| item2 |
| ... | ... |
|trn_5| item7 |


Каждая конректная реализация алгоритма работает со своим форматом входных данных (одним их перечисленных выше), поэтому при использовании сторонних библиотек необходимо предварительно смотреть документацию.

---
---
---

В Sklearn нет функционала для расчета ассоциативных правил, поэтому алгоритм либо реализуется самостоятельно, либо используется сторонняя библиотека.

Попрактикуемся на примере библотеки **mlxtend**

Установка:
- pip install mlxtend
- conda install mlxtend

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

Подготовим данные

In [2]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

Сформируем список товаров, которые втсречались хотя бы раз в наших чеках. Для этого есть удобный класс TransactionEncoder:

In [4]:
te = TransactionEncoder()
te.fit(dataset)

print("Список товаров, которые нашел TransactionEncoder = {}".format(te.columns_))

Список товаров, которые нашел TransactionEncoder = ['Apple', 'Corn', 'Dill', 'Eggs', 'Ice cream', 'Kidney Beans', 'Milk', 'Nutmeg', 'Onion', 'Unicorn', 'Yogurt']


Переведем чеки в табличный формат с числом колонок по числу товаров

In [6]:
transactions_onehot = te.transform(dataset)
transactions_onehot

array([[False, False, False,  True, False,  True,  True,  True,  True,
        False,  True],
       [False, False,  True,  True, False,  True, False,  True,  True,
        False,  True],
       [ True, False, False,  True, False,  True,  True, False, False,
        False, False],
       [False,  True, False, False, False,  True,  True, False, False,
         True,  True],
       [False,  True, False,  True,  True,  True, False, False,  True,
        False, False]])

Для удобства работы обернем нашу табличку в Pandas DataFrame. 

Это пример денормализованного представления транзакционных данных.

In [42]:
df = pd.DataFrame(transactions_onehot, columns=te.columns_)
df

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


Алгоритмом Apriori найдем часто встречающиеся комбинации товаров (frequent itemsets). Мы выводим только те комбинации,  где support > 0.6

Обратите внимание, что такие множества включают и наборы из одного товара!

In [49]:
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.8,(Eggs)
1,1.0,(Kidney Beans)
2,0.6,(Milk)
3,0.6,(Onion)
4,0.6,(Yogurt)
5,0.8,"(Kidney Beans, Eggs)"
6,0.6,"(Eggs, Onion)"
7,0.6,"(Kidney Beans, Milk)"
8,0.6,"(Kidney Beans, Onion)"
9,0.6,"(Yogurt, Kidney Beans)"


Теперь можем найти правила. Так как потенциально их очень много, обязательно ставим ограничение снизу на одну из метрик (confidence, support или lift). В примере ниже ищем правила с confidence > 0.7

In [50]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Kidney Beans),(Eggs),1.0,0.8,0.8,0.80,1.00,0.00,1.000000
1,(Eggs),(Kidney Beans),0.8,1.0,0.8,1.00,1.00,0.00,inf
2,(Onion),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
3,"(Kidney Beans, Eggs)",(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.600000
4,"(Kidney Beans, Onion)",(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf
5,"(Eggs, Onion)",(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
6,(Eggs),"(Kidney Beans, Onion)",0.8,0.6,0.6,0.75,1.25,0.12,1.600000
7,(Onion),"(Kidney Beans, Eggs)",0.6,0.8,0.6,1.00,1.25,0.12,inf
8,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.600000
9,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf


Поясним значение некоторых метрик из вывода выше:
- antecedent_support - частота набора продуктов из левой части правила A
- consequent_support - частота набора продуктов из правой части правила B

Выводятся еще две метрики, которые не описывали, так как они встречаются реже:

**Leverage:**

$leve(A \rightarrow B)= supp(A \rightarrow B) - supp(A) \cdot supp(B) = \frac{|A \cup B|}{|N|} - \frac{|A|}{|N|} \cdot \frac{|B|}{|N|}$

**Conviction:**

$conv(A \rightarrow B) = \frac{1-supp(B)}{1-conf(A \rightarrow B)}$

Суть метрики Conviction - как соотносится популярность других товаров (не B) с частотой ситуации когда наше правило не срабатывает:
- Если conv = 1, то правило бесполезно. Правило ошибается столько же, сколько покупатели выбирают другие товары
- Если conv >> 1, то правило хорошо выделяет закономерность
- Если conv < 1, то правило не просто плохо работает, а выделяет антипаттерн (что никогда не покупают вместе)

Расмотрим правило в строчке 8: Eggs -> Onions.

Прочитаем характеристики данного правила:
- В 60% чеков встречаются яйца и лук (support=0.6)
- 75% тех, кто купил яйца, также купили и лук (confidence=0.75)
- Купившие яйца покупают лук на 25% чаще, чем все остальные покупатели (lift=1.25)
- Купившие яйца покупают другие продукты на 60% реже, чем просто покупающие другие продукты

Полный текст кода

In [54]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

te = TransactionEncoder()
te.fit(dataset)
transactions_onehot = te.transform(dataset)

df = pd.DataFrame(transactions_onehot, columns=te.columns_)

frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets

association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Kidney Beans),(Eggs),1.0,0.8,0.8,0.80,1.00,0.00,1.000000
1,(Eggs),(Kidney Beans),0.8,1.0,0.8,1.00,1.00,0.00,inf
2,(Onion),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
3,"(Kidney Beans, Eggs)",(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.600000
4,"(Kidney Beans, Onion)",(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf
